In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# EDA

## 전체 테이블 Merge -> 분석

In [53]:
orders = pd.read_csv('Data/olist_orders_dataset.csv')
items = pd.read_csv('Data/olist_order_items_dataset.csv')
customers = pd.read_csv('Data/olist_customers_dataset.csv')
reviews = pd.read_csv('Data/olist_order_reviews_dataset.csv')

df = pd.merge(orders, items, on='order_id')
df = pd.merge(df, customers, on='customer_id')
df = pd.merge(df, reviews, on='order_id')

df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

# 1. 배송비 비율 (가격 대비 배송비가 얼마나 차지하는가?)
df['freight_ratio'] = df['freight_value'] / df['price']

# 2. 배송 편차 (예상일 - 실제도착일)
df['arrival_diff'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.days

# 주(State) 단위로 평균 값 집계
state_ranks = df.groupby('customer_state').agg({
    'order_id': 'count',           # 수요 (Demand)
    'freight_ratio': 'mean',       # 평균 배송비 비율
    'arrival_diff': 'mean',        # 평균 배송 편차
    'review_score': 'mean'         # 평균 만족도
}).reset_index()

# 배송비 비율이 높은 지역 Top 5: 배송비 부담이 크므로 '묶음 배송'이나 '무료 배송' 효과가 가장 큰 지역
rank_freight = state_ranks.sort_values(by='freight_ratio', ascending=False).head(5)

print("=== 1. 배송비 비율이 높은 지역 (Bundling Target) ===")
print(rank_freight[['customer_state', 'freight_ratio']])
print("-" * 50)

# 배송 편차가 큰(위험한) 지역 Top 5 : 배송 지연(음수 값)이 심각하므로 '안심 배송일 안내'가 필요한 지역
# arrival_diff가 작을수록(음수일수록) 늦게 오는 것이므로 오름차순 정렬
rank_risk = state_ranks.sort_values(by='arrival_diff', ascending=True).head(5)

print("=== 2. 배송 지연 위험이 큰 지역 (Safe Promise Target) ===")
print(rank_risk[['customer_state', 'arrival_diff']])
print("-" * 50)

# 기회 점수(Gap)가 높은 지역 Top 5 : 수요는 많은데 만족도는 낮아, 진입 시 성공 확률이 높은 지역
# 기회 점수 = (정규화된 수요) * (1 - 정규화된 만족도) -> 수요가 많을수록(1), 만족도가 낮을수록(0 -> 1-0=1) 점수가 높아짐

# 정규화 (0~1 스케일링)
state_ranks['norm_demand'] = (state_ranks['order_id'] - state_ranks['order_id'].min()) / \
                             (state_ranks['order_id'].max() - state_ranks['order_id'].min())

state_ranks['norm_sat'] = (state_ranks['review_score'] - state_ranks['review_score'].min()) / \
                          (state_ranks['review_score'].max() - state_ranks['review_score'].min())

# 기회 점수 계산 (수요 X 불만족도)
state_ranks['opportunity_score'] = state_ranks['norm_demand'] * (1 - state_ranks['norm_sat'])

# 내림차순 정렬 (점수 높은 순)
rank_opportunity = state_ranks.sort_values(by='opportunity_score', ascending=False).head(5)

print("=== 3. 기회 점수가 높은 지역 (Market Entry Target) ===")
print(rank_opportunity[['customer_state', 'order_id', 'review_score', 'opportunity_score']])


=== 1. 배송비 비율이 높은 지역 (Bundling Target) ===
   customer_state  freight_ratio
20             RO       0.593597
21             RR       0.589050
9              MA       0.551003
14             PB       0.513756
19             RN       0.512406
--------------------------------------------------
=== 2. 배송 지연 위험이 큰 지역 (Safe Promise Target) ===
   customer_state  arrival_diff
1              AL      7.841121
24             SE      9.088472
9              MA      9.110138
7              ES      9.820862
4              BA     10.061870
--------------------------------------------------
=== 3. 기회 점수가 높은 지역 (Market Entry Target) ===
   customer_state  order_id  review_score  opportunity_score
18             RJ     14468      3.807092           0.195874
25             SP     47399      4.126859           0.147781
10             MG     13110      4.086957           0.057811
4              BA      3766      3.814392           0.049576
22             RS      6265      4.054270           0.034153
